In [1]:
import pandas as pd
import numpy as np
import logging
from typing import List, Optional

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

In [36]:
import effectsize

In [2]:
dtype_map = pd.read_csv('../outputs/final_df_dtypes.csv', index_col = 0).iloc[:, 0].to_dict()
df = pd.read_csv('../outputs/final_df.csv', dtype = dtype_map)

In [3]:
df.shape

(6461, 149)

In [4]:
df_death = pd.read_csv('../outputs/full_cohort_with_death_data.csv', dtype = dtype_map)

In [8]:
df = pd.merge(df, df_death[['PatientID', 'LineName']], on = 'PatientID', how = 'left')

In [9]:
df.shape

(6461, 150)

In [10]:
df['treatment'] = np.where(df['LineName'] == 'chemo', 0, 1)

In [13]:
df.columns.to_list()

['PatientID',
 'DiseaseGrade',
 'SmokingStatus',
 'Surgery',
 'GroupStage_mod',
 'TStage_mod',
 'NStage_mod',
 'MStage_mod',
 'SurgeryType_mod',
 'days_diagnosis_to_adv',
 'adv_diagnosis_year',
 'days_diagnosis_to_surgery',
 'PrimarySite_lower',
 'Gender',
 'age',
 'Ethnicity_mod',
 'Race_mod',
 'region',
 'PDL1_status',
 'FGFR_status',
 'PDL1_binary',
 'ecog_index',
 'ecog_newly_gte2',
 'weight_index',
 'bmi_index',
 'percent_change_weight',
 'hypotension',
 'tachycardia',
 'fevers',
 'hypoxemia',
 'albumin',
 'alp',
 'alt',
 'ast',
 'bicarbonate',
 'bun',
 'calcium',
 'chloride',
 'creatinine',
 'hemoglobin',
 'platelet',
 'potassium',
 'sodium',
 'total_bilirubin',
 'wbc',
 'albumin_max',
 'alp_max',
 'alt_max',
 'ast_max',
 'bicarbonate_max',
 'bun_max',
 'calcium_max',
 'chloride_max',
 'creatinine_max',
 'hemoglobin_max',
 'platelet_max',
 'potassium_max',
 'sodium_max',
 'total_bilirubin_max',
 'wbc_max',
 'albumin_min',
 'alp_min',
 'alt_min',
 'ast_min',
 'bicarbonate_min',
 '

In [14]:
cat_var = ['ecog_index', 'GroupStage_mod']
cont_var = ['age', 'creatinine', 'albumin']
passthrough_var = ['Surgery', 'ecog_newly_gte2']

In [15]:
# Build pipeline
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler', RobustScaler())
])

categorical_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown = 'ignore'))
])

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_pipeline, cont_var),
        ('cat', categorical_pipeline, cat_var),
        ('pass', 'passthrough', passthrough_var)],
        remainder = 'drop'
)

# Fit and transform
X_preprocessed = preprocessor.fit_transform(df)

In [16]:
# Logistic regression 
lr_model = LogisticRegression(class_weight = 'balanced')
lr_model.fit(X_preprocessed, df['treatment'])

LogisticRegression(class_weight='balanced')

In [18]:
lr_model.predict_proba(X_preprocessed)

array([[0.38171256, 0.61828744],
       [0.53085918, 0.46914082],
       [0.42995375, 0.57004625],
       ...,
       [0.18437828, 0.81562172],
       [0.45429521, 0.54570479],
       [0.75310906, 0.24689094]], shape=(6461, 2))

In [20]:
# Logistic regression 
lr_model = LogisticRegression(class_weight = 'balanced')
lr_model.fit(X_preprocessed, df['treatment'])
propensity_score = lr_model.predict_proba(X_preprocessed)[:, 1]
df['propensity_score'] = propensity_score
        
# Calculate unstabilized weights if stabilized = False
df['weight'] = (
    np.where(df['treatment'] == 1, 
             1/df['propensity_score'], 
             1/(1 - df['propensity_score']))
)

In [22]:
# Get feature names from the preprocessor
feature_names = preprocessor.get_feature_names_out()

# Convert to pandas DataFrame
X_df = pd.DataFrame(X_preprocessed, columns=feature_names, index=df.index)

In [23]:
X_df

,num__age,num__creatinine,num__albumin,cat__ecog_index_0-1,cat__ecog_index_2,cat__ecog_index_3-4,cat__ecog_index_unknown,cat__GroupStage_mod_0-II,cat__GroupStage_mod_III,cat__GroupStage_mod_IV,cat__GroupStage_mod_unknown,pass__Surgery,pass__ecog_newly_gte2
0,0.5,0.0625,0.666667,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0
1,0.285714,-0.041667,0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,0
2,0.571429,0.520833,-0.333333,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0,0
3,0.428571,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0,0
4,-0.428571,-0.208333,1.166667,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6456,-0.714286,0.208333,-0.666667,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0
6457,-1.0,-0.791667,0.666667,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,0
6458,0.785714,-0.625,0.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0
6459,0.0,-0.208333,-1.666667,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0


In [24]:
df

,PatientID,DiseaseGrade,SmokingStatus,Surgery,GroupStage_mod,TStage_mod,NStage_mod,MStage_mod,SurgeryType_mod,days_diagnosis_to_adv,...,lymph_met,thoracic_met,liver_met,bone_met,other_gi_met,other_combined_met,LineName,treatment,propensity_score,weight
0,F0016E985D839,High grade (G2/G3/G4),1,1,IV,T3,N1,M0,upper,0.0,...,0,0,0,0,0,0,Pembrolizumab,1,0.618287,1.617371
1,F001E5D4C6FA0,Low grade (G1),1,1,unknown,T1,unknown,unknown,bladder,274.0,...,0,0,0,0,0,0,chemo,0,0.469141,1.883739
2,F0025BA658AE1,High grade (G2/G3/G4),1,0,unknown,T3,unknown,M0,NaN,621.0,...,0,0,0,0,0,0,Pembrolizumab,1,0.570046,1.754244
3,F00284BE233DB,High grade (G2/G3/G4),1,0,IV,other,unknown,M1,NaN,0.0,...,0,0,0,0,0,1,chemo,0,0.337412,1.509234
4,F0029F9C68879,High grade (G2/G3/G4),1,0,0-II,T2,N0,M0,NaN,170.0,...,0,0,0,0,0,0,Pembrolizumab,1,0.576223,1.735440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6456,FFFEE52179EA7,High grade (G2/G3/G4),1,0,IV,other,unknown,M1,NaN,0.0,...,0,0,0,0,0,0,chemo,0,0.216216,1.275862
6457,FFFEEBF0508C1,High grade (G2/G3/G4),0,1,unknown,other,unknown,unknown,bladder,258.0,...,0,0,0,0,0,0,chemo,0,0.173569,1.210022
6458,FFFF3756D7101,High grade (G2/G3/G4),0,0,0-II,T2,N0,M0,NaN,168.0,...,0,0,0,0,0,0,Atezolizumab,1,0.815622,1.226059
6459,FFFF79E0092F3,High grade (G2/G3/G4),1,1,unknown,other,unknown,unknown,upper,396.0,...,0,0,0,1,0,0,Pembrolizumab,1,0.545705,1.832493


In [29]:
all_var = cat_var + cont_var + passthrough_var

In [30]:
all_var

['ecog_index',
 'GroupStage_mod',
 'age',
 'creatinine',
 'albumin',
 'Surgery',
 'ecog_newly_gte2']

In [31]:
df

,PatientID,DiseaseGrade,SmokingStatus,Surgery,GroupStage_mod,TStage_mod,NStage_mod,MStage_mod,SurgeryType_mod,days_diagnosis_to_adv,...,lymph_met,thoracic_met,liver_met,bone_met,other_gi_met,other_combined_met,LineName,treatment,propensity_score,weight
0,F0016E985D839,High grade (G2/G3/G4),1,1,IV,T3,N1,M0,upper,0.0,...,0,0,0,0,0,0,Pembrolizumab,1,0.618287,1.617371
1,F001E5D4C6FA0,Low grade (G1),1,1,unknown,T1,unknown,unknown,bladder,274.0,...,0,0,0,0,0,0,chemo,0,0.469141,1.883739
2,F0025BA658AE1,High grade (G2/G3/G4),1,0,unknown,T3,unknown,M0,NaN,621.0,...,0,0,0,0,0,0,Pembrolizumab,1,0.570046,1.754244
3,F00284BE233DB,High grade (G2/G3/G4),1,0,IV,other,unknown,M1,NaN,0.0,...,0,0,0,0,0,1,chemo,0,0.337412,1.509234
4,F0029F9C68879,High grade (G2/G3/G4),1,0,0-II,T2,N0,M0,NaN,170.0,...,0,0,0,0,0,0,Pembrolizumab,1,0.576223,1.735440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6456,FFFEE52179EA7,High grade (G2/G3/G4),1,0,IV,other,unknown,M1,NaN,0.0,...,0,0,0,0,0,0,chemo,0,0.216216,1.275862
6457,FFFEEBF0508C1,High grade (G2/G3/G4),0,1,unknown,other,unknown,unknown,bladder,258.0,...,0,0,0,0,0,0,chemo,0,0.173569,1.210022
6458,FFFF3756D7101,High grade (G2/G3/G4),0,0,0-II,T2,N0,M0,NaN,168.0,...,0,0,0,0,0,0,Atezolizumab,1,0.815622,1.226059
6459,FFFF79E0092F3,High grade (G2/G3/G4),1,1,unknown,other,unknown,unknown,upper,396.0,...,0,0,0,1,0,0,Pembrolizumab,1,0.545705,1.832493


In [32]:
smd_df = df[all_var + ['treatment', 'weight']].copy()

In [33]:
smd_df

,ecog_index,GroupStage_mod,age,creatinine,albumin,Surgery,ecog_newly_gte2,treatment,weight
0,2,IV,80,1.23,42.0,1,0,1,1.617371
1,unknown,unknown,77,1.18,41.0,1,0,0,1.883739
2,unknown,unknown,81,1.45,36.0,0,0,1,1.754244
3,unknown,IV,79,NaN,NaN,0,0,0,1.509234
4,0-1,0-II,67,1.10,45.0,0,0,1,1.735440
...,...,...,...,...,...,...,...,...,...
6456,0-1,IV,63,1.30,34.0,0,0,0,1.275862
6457,unknown,unknown,59,0.82,42.0,1,0,0,1.210022
6458,0-1,0-II,84,0.90,41.0,0,0,1,1.226059
6459,0-1,unknown,73,1.10,28.0,1,0,1,1.832493


In [34]:
for var in cont_var:
    smd_df[var] = smd_df[var].fillna(smd_df[var].median())

In [35]:
smd_df

,ecog_index,GroupStage_mod,age,creatinine,albumin,Surgery,ecog_newly_gte2,treatment,weight
0,2,IV,80,1.23,42.0,1,0,1,1.617371
1,unknown,unknown,77,1.18,41.0,1,0,0,1.883739
2,unknown,unknown,81,1.45,36.0,0,0,1,1.754244
3,unknown,IV,79,1.20,38.0,0,0,0,1.509234
4,0-1,0-II,67,1.10,45.0,0,0,1,1.735440
...,...,...,...,...,...,...,...,...,...
6456,0-1,IV,63,1.30,34.0,0,0,0,1.275862
6457,unknown,unknown,59,0.82,42.0,1,0,0,1.210022
6458,0-1,0-II,84,0.90,41.0,0,0,1,1.226059
6459,0-1,unknown,73,1.10,28.0,1,0,1,1.832493


In [39]:
smd_df.dtypes

ecog_index         category
GroupStage_mod     category
age                   Int64
creatinine          float64
albumin             float64
Surgery               Int64
ecog_newly_gte2       Int64
treatment             int64
weight              float64
dtype: object

In [38]:
df_unweighted = effectsize.compute(data = smd_df,
                                   group = 'treatment',
                                   continuous = cont_var,
                                   categorical = cat_var + passthrough_var,
                                   weights = None,
                                   decimals = 3,
                                   intervals = None)

/Users/xavierorcutt/Dropbox/TrialRescue/myenv/lib/python3.13/site-packages/functions.py:264: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  group0 = data.groupby(by = group, axis = 0)[variable].value_counts(normalize = True, sort = False)[0].to_frame().to_numpy()
/Users/xavierorcutt/Dropbox/TrialRescue/myenv/lib/python3.13/site-packages/functions.py:265: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  group1 = data.groupby(by = group, axis = 0)[variable].value_counts(normalize = True, sort = False)[1].to_frame().to_numpy()
/Users/xavierorcutt/Dropbox/TrialRescue/myenv/lib/python3.13/site-packages/functions.py:264: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  group0 = data.groupby(by = group, axis = 0)[variable].value_counts(normalize = True, sort = False)[0].to_frame().to_numpy()
/Users/xavier

TypeError: DataFrame.set_axis() got an unexpected keyword argument 'inplace'

In [ ]:
treatment_col = 'treatment

In [44]:
treat = smd_df[smd_df['treatment'] == 1]

In [ ]:
w_treat = treat['weight']

In [46]:
np.average(treat['creatinine'], weights = treat['weight'])

np.float64(1.3696662531544963)

In [48]:
np.sqrt(np.average((treat['creatinine'] - 1.2) ** 2, weights = treat['weight']))

np.float64(0.7310230847789821)